<a href="https://colab.research.google.com/github/ab-sa/Statistical-Machine-Learning/blob/main/Lecture12_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, GridSearchCV
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
Credit = pd.read_csv('Credit.csv')
Credit['Balance_bin'] = pd.cut(Credit.Balance, bins=[-1,460,2000],labels=['0','1'])
Credit = Credit.drop(['ID', 'Balance', 'Limit', 'Rating'], axis=1)
print('Dimension of the data: ' + str(Credit.shape))
Credit.head()

Dimension of the data: (400, 9)


,Income,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance_bin
0,14.891,2,34,11,Male,No,Yes,Caucasian,0
1,106.025,3,82,15,Female,Yes,Yes,Asian,1
2,104.593,4,71,11,Male,No,No,Asian,1
3,148.924,3,36,11,Female,No,No,Asian,1
4,55.882,2,68,16,Male,No,Yes,Caucasian,0


**Classification**: Fit a KNN with Balance_bin as response:

In [8]:
# data: dropping the credit-card related features (Limit & Rating) to make the classification problem more challenging
X = pd.get_dummies(Credit.drop(['Balance_bin'], axis=1))
y = Credit['Balance_bin']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Tune KNN
knn = KNeighborsClassifier()
k_range = list(range(1, 21))
param_grid = dict(n_neighbors=k_range)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False, verbose=1)

# fitting the model for grid search
grid_search=grid.fit(x_train, y_train)

print(grid_search.best_params_)
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(grid_search.best_score_ *100))

Fitting 10 folds for each of 20 candidates, totalling 200 fits
{'n_neighbors': 10}
Accuracy for our training dataset with tuning is : 63.12%


In [12]:
knn = KNeighborsClassifier(n_neighbors = grid_search.best_params_['n_neighbors']).fit(x_train, y_train)

y_test_pred = knn.predict(x_test) 
test_accuracy=accuracy_score(y_test, y_test_pred)*100
print("Accuracy for our testing dataset with tuning is : {:.2f}%".format(test_accuracy) )

Accuracy for our testing dataset with tuning is : 57.50%


In [13]:
print(confusion_matrix(y_test, y_test_pred, normalize='true'))

[[0.69444444 0.30555556]
 [0.52272727 0.47727273]]
